In [2]:
import pandas as pd
from collections import Counter
import functools
import tqdm
import re
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
dftrain, dftest = pd.read_csv("../train_task1_latest.csv"), pd.read_csv("../test_task1_latest.csv")

In [7]:
dftrain

,paragraph_id,question_id,paragraph,question,target
0,1094,46273,"В отличие от рыб, земноводные (амфибии) и прес...",С какого года Русское Царство перешло на летои...,0.0
1,7414,19164,В 1049 году Балдуину V удалось отнять у Герман...,Кто упомянул о его первых разногласиях со Штей...,0.0
2,6744,39767,Стремление достичь предельных значений ёмкости...,Как называется имеющая мировое значение эпоха ...,0.0
3,7300,36318,Первый практически пригодный двухтактный газов...,Что усугублялось из-за международного давления...,0.0
4,7077,41534,Требуя от художника углубленного изучения изоб...,Какой характер носят пророчества Леонардо да В...,0.0
5,3559,62585,Белки — высокомолекулярные органические вещест...,Какие действия предприняла подводная лодка Чер...,0.0
6,4350,3730,Прайсинговые методы — в основе лежит принцип и...,"Как называют остановки, до которых и на которы...",0.0
7,8012,86629,Применяли изначально для определения близкород...,Какой признак киевский монах Нестор-летописец ...,1.0
8,3634,69421,Успешная конверсия по-разному определяется гру...,"Чтобы понять, по какому рекламному каналу приш...",1.0
9,4350,27335,Прайсинговые методы — в основе лежит принцип и...,Урожайность сои в каких странах почти не отлич...,0.0


In [9]:
dftrain.ix[10,'paragraph']

'Относительно нормативности использовании термина в форме пиксел либо пиксель имеются различные мнения. Так Русский орфографический словарь РАН [4] квалифицирует форму пиксел как общеупотребительную, а форму пиксель как характерную разговорной профессиональной или разговорной и профессиональной речи (в сокращениях словаря нет расшифровки для разг. проф. речи, но есть отдельно разг. — разговорное, проф. — профессиональное[5]; однозначной расшифровки этого определения не даёт и справочная служба русского языка на портале Грамота.ру[6]). С другой стороны, действующий ГОСТ 27459-87[7] предусматривает термин пиксель как единственно возможный для использования в области применения указанного стандарта (компьютерная графика) и который является обязательным для применения в документации и литературе всех видов, входящих в сферу действия стандартизации или использующих результаты этой деятельности . При этом ГОСТ 27459-87 под термином пиксель понимает наименьший элемент поверхности визуализации

In [ ]:
@functools.lru_cache(maxsize=2 ** 19)
def uniq_words(text):
    # Очевидно это все уникальные слова без учета форм
    return set(re.findall("\w+", text))

def calculate_idfs(data):
    counter_paragraph = Counter()
    # Это явно все уникальные параграфы
    uniq_paragraphs = data['paragraph'].unique()
    for paragraph in tqdm.tqdm(uniq_paragraphs, desc="calc idf"):
        set_words = uniq_words(paragraph)
        counter_paragraph.update(set_words)
        
    num_docs = uniq_paragraphs.shape[0]
    idfs = {}
    for word in counter_paragraph:
        idfs[word] = np.log(num_docs / counter_paragraph[word])
    return idfs

In [11]:
idfs = calculate_idfs(dftrain)

calc idf: 100%|██████████| 9078/9078 [00:02<00:00, 4368.86it/s]


In [13]:
# Расчет фич занял 30 минут!
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        question = uniq_words(row.question)
        paragraph = uniq_words(row.paragraph)
        df.loc[index, 'len_paragraph'] = len(paragraph)
        df.loc[index, 'len_question'] = len(question)
        df.loc[index, 'len_intersection'] = len(paragraph & question)
        df.loc[index, 'idf_question'] = np.sum([idfs.get(word, 0.0) for word in question])
        df.loc[index, 'idf_paragraph'] = np.sum([idfs.get(word, 0.0) for word in paragraph])
        df.loc[index, 'idf_intersection'] = np.sum([idfs.get(word, 0.0) for word in paragraph & question])

build features for test: 100%|██████████| 74294/74294 [12:02<00:00, 102.86it/s]


In [6]:
dftrain.head()

,paragraph_id,question_id,paragraph,question,target,len_paragraph,len_question,len_intersection,idf_question,idf_paragraph,idf_intersection
0,5551,56724,31 августа 2016 года АО Банк Финсервис получил...,Кто подтвердил рейтинг кредитоспособности ПАО ...,0.0,65.0,17.0,3.0,83.650717,320.188531,1.446893
1,6678,64727,Лимфатическая система выступает как дополнение...,Какое положение между одиночным и стадным обра...,0.0,70.0,13.0,4.0,64.781264,382.531147,9.737011
2,5941,56747,Тиристорно-импульсная система управления (ТИСУ...,Чем не создается необходимый по величине ток?,1.0,71.0,7.0,5.0,32.478578,389.340211,20.286093
3,2075,50763,После начала борьбы с православием в Литовском...,"В какой части города многочисленны пруды, обра...",0.0,95.0,13.0,3.0,63.039527,532.465918,1.712881
4,5204,53421,В 1954 году Шостакович писал: Дисциплина труда...,К какой культуре по мнению Шнитке принадлежали...,0.0,109.0,10.0,3.0,43.181146,538.020191,11.646477


In [15]:
# Обучение заняло 30 сек
columns = ['len_paragraph', 'len_question', 'len_intersection', 'idf_question', 'idf_paragraph', 'idf_intersection']
model = GradientBoostingClassifier().fit(dftrain[columns], dftrain['target'])
dftest['prediction'] = model.predict_proba(dftest[columns])[:, 1]

In [16]:
dftest[['paragraph_id', 'question_id', 'prediction']].to_csv("prediction2.csv", index=False)